#Facial Recognition for Personalized Hotel Guest Services

**Author**:
ANNEM SAAD

##**ABSTRACT**

Facial recognition technology has emerged as a transformative tool in various sectors, and its application in the hospitality industry promises to revolutionize guest experiences. This research explores the use of facial recognition for personalized hotel guest services, with the aim of streamlining check-in processes, enhancing security, and customizing guest services. The proposed system captures and processes guest images via a webcam interface, storing both facial images and personal information (such as name and room number) in a structured SQLite database. Facial feature detection, including the identification of eyes, smiles, and other key landmarks, is integrated to ensure high recognition accuracy even under varying conditions. Additionally, the system facilitates the linking of guest visual data with their associated profiles, allowing hotels to offer personalized services such as room preferences and tailored amenities based on historical data.

###Section 1: Importing Libraries

Here, we import the required libraries for webcam capture, data handling, GUI rendering, and database operations.

In [2]:
!pip install streamlit

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\OMOLP091\\anaconda3\\Lib\\site-packages\\tenacity\\__init__.py'
Consider using the `--user` option or check the permissions.



     ---------------------------------------- 8.6/8.6 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 6.9/6.9 MB 5.9 MB/s eta 0:00:00
     -------------------------------------- 731.2/731.2 kB 5.8 MB/s eta 0:00:00
     -------------------------------------- 233.8/233.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Rolling back uninstall of tenacity
  Moving to c:\users\omolp091\anaconda3\lib\site-packages\tenacity-8.0.1.dist-info\
   from C:\Users\OMOLP091\anaconda3\Lib\site-packages\~-nacity-8.0.1.dist-info
  Moving to c:\users\omolp091\anaconda3\lib\site-packages\tenacity\
   from C:\Users\OMOLP091\anaconda3\Lib\site-packages\~-nacity
^C


In [12]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import streamlit as st
import cv2
import os
import sqlite3


###Section 2: Webcam Image Capture Function

This section defines a function to capture an image using the webcam and save it to a specified location.

In [1]:
# Function to capture and save an image using the webcam
def capture_image(save_path):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        st.error("Error: Could not open webcam")
        return

    st.write("Press 'Space' to Capture")

    while True:
        ret, frame = cap.read()
        if not ret:
            st.error("Error: Could not read frame")
            break

        cv2.imshow("Webcam Feed - Press space to capture", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord(" "):
            save_file = os.path.join(save_path, "captured_image.jpg")
            cv2.imwrite(save_file, frame)
            st.success(f"Image saved to {save_file}")
            break
        elif key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
print("end of function")


end of function


###Section 3: Database Connection Function

This section defines a function to establish a connection to the SQLite database.

In [2]:
# Function to create a connection to SQLite database
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to database: {db_file}")
    except sqlite3.Error as e:
        print(e)
    return conn
print("end of function")


end of function


###Section 4: Database Table Creation Function

Here, we define a function to create a table in the database if it doesn't already exist.

In [4]:
# Function to create a table in the database if it doesn't already exist
def create_table(conn):
    try:
        query = '''CREATE TABLE IF NOT EXISTS user_data (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT NOT NULL,
                        age INTEGER,
                        email TEXT
                    );'''
        conn.execute(query)
        print("Table created successfully (if it didn't exist).")
    except sqlite3.Error as e:
        print(e)
print("end of function")

end of function


###Section 5: Data Insertion into Database

This function handles inserting user data into the database.

In [5]:
# Function to insert data into the database
def insert_data(conn, user_data):
    try:
        query = '''INSERT INTO user_data (name, age, email)
                   VALUES (?, ?, ?);'''
        conn.execute(query, user_data)
        conn.commit()  # Commit the transaction
        print("Data inserted successfully!")
    except sqlite3.Error as e:
        print(e)
print("end of function")

end of function


###Section 6: Haar Cascade Classification

This section detects features such as faces, eyes, and smiles using Haar cascades.

In [6]:
# Haar cascade classification function
def classify_and_save_features(face_cascade_path, eye_cascade_path, smile_cascade_path):
    # Check if cascade files exist
    for path in [face_cascade_path, eye_cascade_path, smile_cascade_path]:
        if not os.path.isfile(path):
            raise FileNotFoundError(f"Cascade file not found at {path}")

    # Load cascades
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    eye_cascade = cv2.CascadeClassifier(eye_cascade_path)
    smile_cascade = cv2.CascadeClassifier(smile_cascade_path)

    cap = cv2.VideoCapture(0)
    features_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

            # Detect and mark eyes
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 10)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            # Detect and mark smiles
            smiles = smile_cascade.detectMultiScale(roi_gray, 1.7, 22)
            for (sx, sy, sw, sh) in smiles:
                cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 2)

        cv2.imshow("Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
print("end of function")

end of function


###Section 7: Bill Generation Placeholder

This is a placeholder for the bill generation feature.

In [7]:
# Placeholder for bill generation logic
def generate_bill(name):
    st.write(f"Bill generation logic for {name} goes here.")
print("end of function")

end of function


###Section 8: Streamlit Main Application

This section integrates all functionalities into the Streamlit app.

In [ ]:
# Main function to run the Streamlit app
def main():
    st.title("Webcam Image Capture and Database Storage")

    if st.button("Click Image"):
        save_path = r"./captured_images"
        os.makedirs(save_path, exist_ok=True)
        capture_image(save_path)

    person_name = st.text_input("Write the Name of the Customer")
    if st.button("Store in Database"):
        database = "test.db"
        conn = create_connection(database)
        if conn:
            create_table(conn)
            age = st.number_input("Enter Age", min_value=0, max_value=120, step=1)
            email = st.text_input("Enter Email")
            if st.button("Insert Data"):
                insert_data(conn, (person_name, age, email))
            conn.close()

    if st.button("Classify"):
        classify_and_save_features('./haarcascade_frontalface_default.xml',
                                   './haarcascade_eye.xml',
                                   './haarcascade_smile.xml')

    name = st.text_input("Name for Bill Generation")
    if st.button("Generate Bill"):
        generate_bill(name)
print("end of function1")
if __name__ == "__main__":
    main()
print("end of function2")

end of function1


2024-12-05 17:11:40.970 
  command:

    streamlit run C:\Users\saad2\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-05 17:11:40.972 Session state does not function when running a script without `streamlit run`


end of function2


###Section 9: Run the Application

Run the Streamlit app in this final section.

In [ ]:
!streamlit run C:\Users\saad2\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py 

#ENHANCEMENTS

In [ ]:
import numpy as np
import pandas as pd
import streamlit as st
import cv2
import os
import sqlite3

# ENHANCEMENT # 1:Input Fields for Additional Guest Information

Add Input Fields for Additional Guest Information
Extend the form to collect more details during check-in, such as dietary preferences, arrival/departure times, and preferred languages.*italicized text*

In [14]:
### Section 1: Input Fields for Additional Guest Information
# This section collects additional details like dietary preferences and arrival/departure times.

def collect_additional_info():
    st.subheader("Additional Guest Information")
    dietary_preferences = st.text_input("Dietary Preferences (e.g., Vegetarian, Vegan, etc.)")
    arrival_time = st.time_input("Arrival Time")
    departure_time = st.time_input("Departure Time")
    preferred_language = st.text_input("Preferred Language")

    return dietary_preferences, arrival_time, departure_time, preferred_language
print("end of function")

end of function


# ENHANCEMENT # 2:Guest Visit Purpose and Preferences


Include input fields to capture the guest’s purpose of stay (e.g., business, vacation) and any specific preferences like quiet rooms or accessibility features. This helps staff better tailor their services.

In [15]:
### Section 2: Guest Visit Purpose and Preferences
# Collects information about the guest’s purpose of stay and specific preferences.

def collect_visit_purpose():
    st.subheader("Guest Visit Purpose and Preferences")
    visit_purpose = st.selectbox("Purpose of Stay", ["Business", "Vacation", "Others"])
    room_preference = st.selectbox("Room Preferences", ["Quiet Room", "Accessibility Features", "Any"])

    return visit_purpose, room_preference
print("end of function")

end of function


# ENHANCEMENT # 3: Basic Billing Information Storage


Add input fields to capture expenses like meals, spa treatments, or minibar usage. This allows you to use the "Generate Bill" button to create a simple breakdown of costs, improving the guest checkout experience.



In [16]:
### Section 3: Basic Billing Information Storage
# Captures expenses like meals, spa treatments, minibar usage, and stores them for bill generation.

def collect_billing_info():
    st.subheader("Basic Billing Information")
    meals_expense = st.number_input("Meals Expense ($)", min_value=0.0, step=0.01)
    spa_expense = st.number_input("Spa Treatments Expense ($)", min_value=0.0, step=0.01)
    minibar_expense = st.number_input("Minibar Usage Expense ($)", min_value=0.0, step=0.01)

    total_expense = meals_expense + spa_expense + minibar_expense
    st.write(f"Total Expenses: ${total_expense}")

    return meals_expense, spa_expense, minibar_expense, total_expense
print("end of function")

end of function


# ENHANCEMENT # 4:Notes Section

Include a free-text "Additional Notes" field where staff can record special requests or important details about the guest, such as anniversaries or specific instructions.



In [17]:
### Section 4: Notes Section
# A free-text field where staff can record special requests or other important details about the guest.

def additional_notes():
    st.subheader("Additional Notes")
    notes = st.text_area("Enter any special requests or important details about the guest.")
    return notes
print("end of function")

end of function


# ENHANCEMENT # 5:Confirmation Message After Data Entry

Display a clear confirmation message or summary after the data has been stored successfully, allowing staff to review the details and ensuring data entry accuracy.






In [18]:
### Section 5: Confirmation Message After Data Entry
# Displays a summary of the entered data for review and confirmation.

def display_confirmation(name, dietary_preferences, arrival_time, departure_time, preferred_language, visit_purpose, room_preference, meals_expense, spa_expense, minibar_expense, notes):
    st.subheader("Confirmation of Data Entry")
    st.write(f"Name: {name}")
    st.write(f"Dietary Preferences: {dietary_preferences}")
    st.write(f"Arrival Time: {arrival_time}")
    st.write(f"Departure Time: {departure_time}")
    st.write(f"Preferred Language: {preferred_language}")
    st.write(f"Visit Purpose: {visit_purpose}")
    st.write(f"Room Preference: {room_preference}")
    st.write(f"Meals Expense: ${meals_expense}")
    st.write(f"Spa Expense: ${spa_expense}")
    st.write(f"Minibar Expense: ${minibar_expense}")
    st.write(f"Additional Notes: {notes}")

    if st.button("Confirm and Save"):
        st.success("Data successfully saved!")
    else:
        st.warning("Please review and confirm the details.")



# Main Application

In [19]:
### Streamlit Main Application
def main():
    st.title("Enhanced Hotel Guest Services")

    person_name = st.text_input("Write the Name of the Customer")

    # Collect guest details
    dietary_preferences, arrival_time, departure_time, preferred_language = collect_additional_info()
    visit_purpose, room_preference = collect_visit_purpose()
    meals_expense, spa_expense, minibar_expense, total_expense = collect_billing_info()
    notes = additional_notes()

    # Display the confirmation message
    display_confirmation(person_name, dietary_preferences, arrival_time, departure_time, preferred_language, visit_purpose, room_preference, meals_expense, spa_expense, minibar_expense, notes)

    if st.button("Store in Database"):
        database = "test.db"
        conn = create_connection(database)
        if conn:
            create_table(conn)
            age = st.number_input("Enter Age", min_value=0, max_value=120, step=1)
            email = st.text_input("Enter Email")
            if st.button("Insert Data"):
                insert_data(conn, (person_name, age, email))
            conn.close()

---
---
